In [1]:
import tensorflow as tf
import numpy as np

# Example word: "XMAS"
char_to_num = {'X': 1, 'M': 2, 'A': 3, 'S': 4}
target_sum = sum(char_to_num[c] for c in "XMAS")  # 10

def read_grid_from_file(filename):
    with open(filename, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    grid = np.array([list(line) for line in lines], dtype='<U1')
    return grid

def grid_to_tensor(grid):
    # Convert grid of chars to numeric tensor
    numeric_grid = np.zeros(grid.shape, dtype=np.float32)
    for c, val in char_to_num.items():
        numeric_grid[grid == c] = val
    # Add batch and channel dimension: [1, H, W, 1]
    return tf.constant(numeric_grid[None, :, :, None], dtype=tf.float32)

def create_kernels():
    # We'll create 8 filters for "XMAS" and "SAMX" in all directions:
    # Directions: horizontal, vertical, diagonal, anti-diagonal
    # Patterns: XMAS (forward), SAMX (reverse)
    # Each kernel is placed so that convolution at that point = sum(X,M,A,S)=10 if matched.

    # We'll represent kernels as numpy arrays, then combine into a single tf.constant.

    # For horizontal (XMAS): shape [1,4]
    xmas_h = np.array([1,2,3,4], dtype=np.float32).reshape(1,4,1,1)  # kernel_h: [1,4,1,1]
    samx_h = np.array([4,3,2,1], dtype=np.float32).reshape(1,4,1,1)  # reverse horizontal

    # For vertical (XMAS): shape [4,1]
    xmas_v = np.array([1,2,3,4], dtype=np.float32).reshape(4,1,1,1)
    samx_v = np.array([4,3,2,1], dtype=np.float32).reshape(4,1,1,1)

    # For diagonal (XMAS): shape [4,4], X=1 at [0,0], M=2 at [1,1], A=3 at [2,2], S=4 at [3,3]
    xmas_d = np.zeros((4,4), dtype=np.float32)
    np.fill_diagonal(xmas_d, [1,2,3,4])
    xmas_d = xmas_d.reshape(4,4,1,1)

    # SAMX diagonal (reverse), S=4 at [0,0], A=3 at [1,1], M=2 at [2,2], X=1 at [3,3]
    samx_d = np.zeros((4,4), dtype=np.float32)
    np.fill_diagonal(samx_d, [4,3,2,1])
    samx_d = samx_d.reshape(4,4,1,1)

    # For anti-diagonal (XMAS): 
    # X at [0,3], M at [1,2], A at [2,1], S at [3,0]
    xmas_ad = np.zeros((4,4), dtype=np.float32)
    xmas_ad[0,3] = 1
    xmas_ad[1,2] = 2
    xmas_ad[2,1] = 3
    xmas_ad[3,0] = 4
    xmas_ad = xmas_ad.reshape(4,4,1,1)

    # SAMX anti-diagonal:
    # S at [0,3], A at [1,2], M at [2,1], X at [3,0]
    samx_ad = np.zeros((4,4), dtype=np.float32)
    samx_ad[0,3] = 4
    samx_ad[1,2] = 3
    samx_ad[2,1] = 2
    samx_ad[3,0] = 1
    samx_ad = samx_ad.reshape(4,4,1,1)

    # Combine all filters along the last dimension
    # We have 8 filters total:
    # [xmas_h, samx_h, xmas_v, samx_v, xmas_d, samx_d, xmas_ad, samx_ad]
    kernels = np.concatenate([xmas_h, samx_h, xmas_v, samx_v, xmas_d, samx_d, xmas_ad, samx_ad], axis=3)

    return tf.constant(kernels, dtype=tf.float32)

def count_occurrences(grid_tensor):
    # grid_tensor: [1, H, W, 1]
    kernels = create_kernels()
    # Perform convolution: out shape [1, H', W', 8]
    # Pad='VALID' so that we only match fully within bounds.
    out = tf.nn.conv2d(grid_tensor, kernels, strides=[1,1,1,1], padding='VALID')
    # out: [1, H', W', 8], each channel corresponds to one pattern

    # Check where out equals target_sum (10). Because these are perfect integer sums, 
    # we can use a tolerance or exact comparison.
    # Count how many times any filter channel matches target_sum.
    matches = tf.reduce_sum(tf.cast(tf.equal(out, target_sum), tf.int32))
    return matches.numpy()

if __name__ == "__main__":
    # Assume 'wordsearch.txt' is the puzzle file
    grid = read_grid_from_file("test.txt")
    grid_tensor = grid_to_tensor(grid)
    total = count_occurrences(grid_tensor)
    print("Total occurrences of 'XMAS':", total)


ModuleNotFoundError: No module named 'tensorflow'